In [1]:
import traceback
import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [2]:
# 指定工作空间
env.workspace = r'D:\ArcGISProjects\workspace\shbyq\DZ.gdb'
env.overwriteOutput = True

In [3]:
def merge_small_parcels(input_fc, output_fc, land_type_field, dz_field, thresholds, batch_size=1000):
    try:
        arcpy.CopyFeatures_management(input_fc, output_fc)
        print(f"已创建输出要素类: {output_fc}")

        arcpy.AddSpatialIndex_management(output_fc)  
        print("已添加空间索引")

        if "Area" not in [f.name for f in arcpy.ListFields(output_fc)]:
            arcpy.AddField_management(output_fc, "Area", "DOUBLE")
            print("已添加 Area 字段")

        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
        print("已计算面积")

        arcpy.MakeFeatureLayer_management(output_fc, "output_layer")

        small_parcels = []
        with arcpy.da.SearchCursor(output_fc, ["OID@", land_type_field, dz_field, "Area", "SHAPE@"]) as cursor:
            for row in cursor:
                oid, land_type, dz, area, shape = row
                threshold = thresholds.get(dz, min(thresholds.values()))
                if area < threshold:
                    small_parcels.append((oid, land_type, dz, area, shape))
        print(f"找到 {len(small_parcels)} 个小于指定阈值的图斑") 

        merged_count = 0
        total_parcels = len(small_parcels)
        processed_parcels = 0

        small_parcels_batch = []
        for small_parcel in small_parcels:
            small_parcels_batch.append(small_parcel)

            if len(small_parcels_batch) == batch_size:
                merged_count += process_batch(small_parcels_batch, output_fc, "output_layer", land_type_field)
                processed_parcels += len(small_parcels_batch)
                print(f"已处理 {processed_parcels}/{total_parcels} 个图斑")
                small_parcels_batch = []

        if small_parcels_batch:
            merged_count += process_batch(small_parcels_batch, output_fc, "output_layer", land_type_field)
            processed_parcels += len(small_parcels_batch)
            print(f"已处理 {processed_parcels}/{total_parcels} 个图斑")

        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")

        print(f"操作完成。合并了 {merged_count} 个小面积图斑到相邻的相同地类图斑中。")
        print(f"结果保存在新的要素类: {output_fc}") 

    except Exception as e:
        print(f"发生错误: {str(e)}")
        print(traceback.format_exc())


def process_batch(small_parcels_batch, output_fc, layer_name, land_type_field):
    merged_count = 0
    total_parcels_batch = len(small_parcels_batch)
    processed_parcels_batch = 0

    edit = arcpy.da.Editor(arcpy.Describe(output_fc).path)
    edit.startEditing(False, False)
    edit.startOperation()

    try:
        land_type_field_object = arcpy.ListFields(layer_name, land_type_field)[0]

        for small_parcel in small_parcels_batch:
            oid, land_type, dz, area, shape = small_parcel

            arcpy.SelectLayerByLocation_management(layer_name, "BOUNDARY_TOUCHES", shape)

            max_area = 0
            max_oid = None

            with arcpy.da.SearchCursor(layer_name, ["OID@", "Area", land_type_field_object.name]) as neighbor_cursor:
                for neighbor in neighbor_cursor:
                    if neighbor[0] != oid and neighbor[2] == land_type and neighbor[1] > max_area:
                        max_area = neighbor[1]
                        max_oid = neighbor[0]

            if max_oid:
                try:
                    where_clause = f"OBJECTID IN ({oid}, {max_oid})"
                    with arcpy.da.UpdateCursor(output_fc, ["OID@", "SHAPE@", land_type_field_object.name], where_clause) as update_cursor:
                        shapes = []
                        for update_row in update_cursor:
                            shapes.append(update_row[1])
                            if update_row[0] == oid:
                                update_cursor.deleteRow()

                        if len(shapes) == 2:
                            merged_shape = shapes[0].union(shapes[1])
                            update_cursor.reset()
                            for update_row in update_cursor:
                                if update_row[0] == max_oid:
                                    update_cursor.updateRow([max_oid, merged_shape.convexHull(), land_type])
                                    merged_count += 1
                                    break

                except Exception as e:
                    print(f"处理 OID {oid} 时出错: {str(e)}")
                    print(traceback.format_exc())
            
            processed_parcels_batch += 1
            print(f"当前批次已处理 {processed_parcels_batch}/{total_parcels_batch} 个小图斑")

            arcpy.SelectLayerByAttribute_management(layer_name, "CLEAR_SELECTION")
        
        edit.stopOperation()
        edit.stopEditing(True)
        
    except Exception as e:
        edit.abortOperation()
        edit.stopEditing(False)
        raise e
    
    finally:
        if edit.isEditing:
            edit.stopEditing(True)

    return merged_count

In [4]:
# 使用示例
if __name__ == "__main__":
    input_fc = r"D:\ArcGISProjects\workspace\shbyq\DZ.gdb\DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_COPY"
    output_fc = "DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_COPY_END"
    land_type_field = "DLMC"
    dz_field = "DZ"
    thresholds = {
        "01": 50,
        "03": 1000,
        "04": 1000,
        # 可以继续添加其他DZ值的阈值
    }

    merge_small_parcels(input_fc, output_fc, land_type_field, dz_field, thresholds)
# 75271

当前批次已处理 217/1000 个小图斑
当前批次已处理 218/1000 个小图斑
当前批次已处理 219/1000 个小图斑
当前批次已处理 220/1000 个小图斑
当前批次已处理 221/1000 个小图斑
当前批次已处理 222/1000 个小图斑
当前批次已处理 223/1000 个小图斑
当前批次已处理 224/1000 个小图斑
当前批次已处理 225/1000 个小图斑
当前批次已处理 226/1000 个小图斑
当前批次已处理 227/1000 个小图斑
当前批次已处理 228/1000 个小图斑
当前批次已处理 229/1000 个小图斑
当前批次已处理 230/1000 个小图斑
当前批次已处理 231/1000 个小图斑
当前批次已处理 232/1000 个小图斑
当前批次已处理 233/1000 个小图斑
当前批次已处理 234/1000 个小图斑
当前批次已处理 235/1000 个小图斑
当前批次已处理 236/1000 个小图斑
当前批次已处理 237/1000 个小图斑
当前批次已处理 238/1000 个小图斑
当前批次已处理 239/1000 个小图斑
当前批次已处理 240/1000 个小图斑
当前批次已处理 241/1000 个小图斑
当前批次已处理 242/1000 个小图斑
当前批次已处理 243/1000 个小图斑
当前批次已处理 244/1000 个小图斑
当前批次已处理 245/1000 个小图斑
当前批次已处理 246/1000 个小图斑
当前批次已处理 247/1000 个小图斑
当前批次已处理 248/1000 个小图斑
当前批次已处理 249/1000 个小图斑
当前批次已处理 250/1000 个小图斑
当前批次已处理 251/1000 个小图斑
当前批次已处理 252/1000 个小图斑
当前批次已处理 253/1000 个小图斑
当前批次已处理 254/1000 个小图斑
当前批次已处理 255/1000 个小图斑
当前批次已处理 256/1000 个小图斑
当前批次已处理 257/1000 个小图斑
当前批次已处理 258/1000 个小图斑
当前批次已处理 259/1000 个小图斑
当前批次已处理 260/1000 个小图斑
当前批次已处理 261/1000 个小图斑
当前批次已处理 26